In [ ]:
import numpy as np
import os
import pandas as pd

from datetime import datetime, timedelta
from dotenv import load_dotenv

import hvplot.pandas
 
from alpaca.data.requests import StockBarsRequest
from alpaca.data.historical.stock import StockHistoricalDataClient
from  alpaca.data.timeframe import TimeFrame, TimeFrameUnit

In [ ]:
load_dotenv()

In [ ]:
ALPACA_API_KEY = os.getenv("ALPACA_API_KEY")
ALPACA_SECRET_KEY = os.getenv("ALPACA_API_SECRET")

In [ ]:
client = StockHistoricalDataClient(ALPACA_API_KEY, ALPACA_SECRET_KEY)

In [ ]:
timeframe = TimeFrame(5, TimeFrameUnit.Minute)
symbol = 'TSLA'
start = datetime.utcnow() - timedelta(days=30)
end=datetime.utcnow() - timedelta(days=1)
request = StockBarsRequest(symbol_or_symbols=symbol, start=start, end=end, timeframe=timeframe)

In [ ]:
source_df = client.get_stock_bars(request).df.tz_convert('America/New_York', level=1)

In [ ]:
display(source_df.tail())

In [ ]:
bars_df = source_df.copy()
bars_df = bars_df.reset_index(level=[0]).drop(columns=["symbol"])
display(bars_df)

In [ ]:


# pct_change is profit from last close
bars_df["pct_change"] = bars_df["close"].pct_change()
# signal for when we want to be in or out of a stock
bars_df["signal"] = np.where(bars_df["pct_change"] > 0, 1.0, 0.0)
# reaction is the signal diff
bars_df["reaction"] = bars_df["signal"].diff()
# action is if we could perfectly predict the next close
bars_df["action"] = bars_df["reaction"].shift(-1)
# these values are the high, low, and open as a percentage of the current close
bars_df["high %"] = (bars_df["high"] - bars_df["close"])/bars_df["close"]
bars_df["low %"] = (bars_df["low"] - bars_df["close"])/bars_df["close"]
bars_df["open %"] = (bars_df["open"] - bars_df["close"])/bars_df["close"]

In [ ]:
display(bars_df.head(100))
display(bars_df.tail(100))

In [ ]:
# So we want to add the high, low, and open for the 15 minute and 60 minute window
minutes = 5
windows = [3, 12]

In [ ]:
for window in windows:
    wm = window*minutes
    # compute the rolling high, low, open for a given window.  the close is the same at this time for all windows
    bars_df[f"high - {wm}"] = bars_df["high"].rolling(window=window).max()
    bars_df[f"low - {wm}"] = bars_df["low"].rolling(window=window).min()
    bars_df[f"open - {wm}"] = bars_df["open"].shift(periods=window-1)
    # these values are the high, low, and open as a percentage of the current close
    bars_df[f"high % - {wm}"] = (bars_df[f"high - {wm}"] - bars_df["close"])/bars_df["close"]
    bars_df[f"low % - {wm}"] = (bars_df[f"low - {wm}"] - bars_df["close"])/bars_df["close"]
    bars_df[f"open % - {wm}"] = (bars_df[f"open - {wm}"] - bars_df["close"])/bars_df["close"]

In [ ]:
display(bars_df.head(100))
display(bars_df.tail(100))